In [2]:
#NLTK and Other Libraries
import os
import io
import re
import itertools
import operator
from collections import Counter
from langdetect import detect
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
import numpy as np
import spacy
from difflib import SequenceMatcher
import en_core_web_sm
nlp = en_core_web_sm.load()

#Data Visualization Libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.markers import MarkerStyle
import seaborn as sns

#Date time Libraries
import time
import datetime

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingadhar_duggirala\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


In [166]:
sec_metadata=pd.read_excel("C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\Sample Metadata SEC_Final.xlsx")

In [162]:
sec_metadata["Has Income Statement Flag"].value_counts()

True     1939
False    1132
Name: Has Income Statement Flag, dtype: int64

In [175]:
sec_metadata.PeriodType.value_counts()

Quarter       2073
Annual         877
SemiAnnual      22
Name: PeriodType, dtype: int64

In [180]:
a=sec_metadata[(sec_metadata["Has Income Statement Flag"]==False)&(sec_metadata.PeriodType.isin(["Annual","Quarter","SemiAnnual"]))]

In [184]:
sec_metadata_final=sec_metadata.merge(a, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

In [201]:
sec_metadata_final[~sec_metadata_final["PeriodType"].isna()]["Has Income Statement Flag"].

array([ True])

In [185]:
sec_metadata_final["Has Income Statement Flag"].value_counts()

True     1939
False      99
Name: Has Income Statement Flag, dtype: int64

In [13]:
sec_metadata[sec_metadata.VersionId==97924048]

,DocumentId,doc cnt,VersionId,CompanyId,Company Name,documentToObjectRelId,elementGroupId,PeriodOfReport,Language,FilingDate,FilingDateSource,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Is Errata Amendment Flag,Has Notes Flag,Has Audited Financials Flag,Category
0,322994151,4,97924048,24927252,Noranda Alumina LLC,347206249,-1295509969,2009-08-31,English,2010-05-14 16:28:42,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,0.0,VALIDATION
1,322994151,4,97924048,24927252,Noranda Alumina LLC,347206249,-1295509965,2008-12-31,English,2010-05-14 16:28:42,Filing date on the website,Annual,True,0.0,0.0,0.0,1.0,1.0,TRAIN
2,322994151,4,97924048,34074951,Noranda Aluminum Holding Corporation,82561871,-2143476676,2010-03-31,English,2010-05-14 16:28:42,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,0.0,VALIDATION
3,322994151,4,97924048,34074951,Noranda Aluminum Holding Corporation,82561871,363713659,2009-12-31,English,2010-05-14 16:28:42,Filing date on the website,Annual,True,0.0,0.0,0.0,1.0,1.0,TRAIN


In [14]:
sec_metadata.columns

Index(['DocumentId', 'doc cnt', 'VersionId', 'CompanyId', 'Company Name',
       'documentToObjectRelId', 'elementGroupId', 'PeriodOfReport', 'Language',
       'FilingDate', 'FilingDateSource', 'PeriodType',
       'Has Income Statement Flag', 'Is Preliminary Flag', 'Is Amendment Flag',
       'Is Errata Amendment Flag', 'Has Notes Flag',
       'Has Audited Financials Flag', 'Category'],
      dtype='object')

In [164]:
sec_metadata_final[sec_metadata_final.VersionId==491529222]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
577,491529222,45842867,"Business First Bancshares, Inc.",2017-12-31,Annual,True,1.0,0.0,NaN
579,491529222,45842867,"Business First Bancshares, Inc.",2018-06-30,Quarter,True,1.0,0.0,NaN


In [186]:
sec_metadata_final["Has Income Statement Flag"].value_counts()

True     1939
False      99
Name: Has Income Statement Flag, dtype: int64

In [96]:
sec_metadata_final.shape

(2946, 9)

In [29]:
sec_metadata_final[sec_metadata_final.VersionId==259118202]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
653,259118202,25070108,Orpak Systems Ltd.,NaT,NaN,False,NaN,NaN,NaN
654,259118202,318505463,Fortive Corporation,2017-12-31,Annual,True,0.0,0.0,1.0


In [30]:
sec_metadata_final[sec_metadata_final.VersionId==230292662]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
645,230292662,332358,"Mid-America Apartment Communities, Inc.",2017-12-31,Annual,True,1.0,0.0,NaN
646,230292662,332358,"Mid-America Apartment Communities, Inc.",2017-12-31,Quarter,True,1.0,0.0,NaN
647,230292662,27525007,"Mid-America Apartments, L.P.",NaT,NaN,False,NaN,NaN,NaN


In [58]:
sec_metadata_final[sec_metadata_final.VersionId==230292662]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
645,230292662,332358,"Mid-America Apartment Communities, Inc.",2017-12-31,Annual,True,1.0,0.0,NaN
646,230292662,332358,"Mid-America Apartment Communities, Inc.",2017-12-31,Quarter,True,1.0,0.0,NaN


In [31]:
sec_metadata_final[sec_metadata_final.VersionId==285661030]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
668,285661030,174901,Dover Corporation,2018-03-22,Quarter,False,0.0,1.0,NaN
669,285661030,545985501,Apergy Corporation,2018-05-04,Quarter,False,0.0,0.0,NaN


In [249]:
k=0
onlyfiles = [f for f in listdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN') if isfile(join('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN', f))]
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN')
mv_df_inc=pd.DataFrame(columns=(['versionid',"companyname","incomeflag"]))
for file in onlyfiles:
    lin=""
    print(file)
    a=file
    mv_df_inc.loc[k,['versionid']]=os.path.splitext(a)[0]
    with open(file,encoding="utf8") as input_data:
        for line in input_data:
            mv_df_inc.loc[k,['companyname']]=line.split(" ",1)[1].strip().split("  ",1)[0]
            break
        for line in input_data:
            if any(x in line.lower() for x in ["consolidated statements","consolidated statement","condensed","statement","data"]):
                if any(x in line.lower() for x in ["operations","income","financial","earnings"]):
                    if line.strip()[0].isupper():
                        #if line.strip().istitle():
                        lin+=line
    if len(lin)==0:
        mv_df_inc.loc[k,['incomeflag']]=False
        k+=1
    else:
        mv_df_inc.loc[k,['incomeflag']]=True
        k+=1        

175605384.txt
209271852.txt
226135784.txt
226157588.txt
230288860.txt
230292662.txt
232460148.txt
236972360.txt
237114582.txt
237143018.txt
239593846.txt
242236708.txt
242607122.txt
243576070.txt
243724800.txt
244735926.txt
244842314.txt
246273214.txt
249898752.txt
250984690.txt
252567172.txt
252688048.txt
253498342.txt
253651946.txt
253715340.txt
253827726.txt
253835662.txt
253874620.txt
253891268.txt
256166326.txt
256378252.txt
256468776.txt
257505788.txt
258140358.txt
258228198.txt
258264294.txt
258370180.txt
259118202.txt
259585392.txt
259602418.txt
259720618.txt
259844898.txt
259903996.txt
260833248.txt
260845914.txt
261066672.txt
261273360.txt
261556614.txt
261655748.txt
261699460.txt
261699932.txt
261757808.txt
263058312.txt
263081942.txt
264921594.txt
267882938.txt
270663546.txt
273075380.txt
273186958.txt
275798274.txt
278984340.txt
279035452.txt
281045440.txt
282977390.txt
285534432.txt
285661030.txt
291242490.txt
291393110.txt
291449076.txt
292669118.txt
300835588.txt
301798

471338908.txt
471339086.txt
471339880.txt
471340572.txt
471340594.txt
471341822.txt
471342308.txt
471342868.txt
471342954.txt
471343010.txt
471343498.txt
471343984.txt
471346522.txt
471348592.txt
471348794.txt
471349672.txt
471349678.txt
471350510.txt
471351264.txt
471351616.txt
471352770.txt
471352954.txt
471353888.txt
471356462.txt
471358032.txt
471358164.txt
471367136.txt
471367146.txt
471367194.txt
471367494.txt
471368794.txt
471369252.txt
471370952.txt
471373380.txt
471438356.txt
471438562.txt
471439452.txt
471439642.txt
471439654.txt
471439732.txt
471439740.txt
471440742.txt
471444576.txt
471444584.txt
471444594.txt
471444786.txt
471722940.txt
471724936.txt
471725320.txt
471725516.txt
471725606.txt
471727162.txt
471727408.txt
471727630.txt
471727786.txt
471727788.txt
471729450.txt
471729850.txt
471736558.txt
471740776.txt
471775728.txt
471779360.txt
471779652.txt
471781914.txt
471785430.txt
471786042.txt
471798508.txt
471800736.txt
471800780.txt
471811774.txt
471812656.txt
471814

477746362.txt
477749356.txt
477779596.txt
477782276.txt
477784694.txt
477785082.txt
477785878.txt
477787830.txt
477797468.txt
477810762.txt
477814142.txt
477869622.txt
477887590.txt
477896794.txt
477900588.txt
477914856.txt
477943216.txt
477993898.txt
478003336.txt
478009080.txt
478016144.txt
478016784.txt
478033228.txt
478036416.txt
478050230.txt
478063634.txt
478063722.txt
478063786.txt
478066258.txt
478066522.txt
478069716.txt
478070760.txt
478072374.txt
478072532.txt
478074270.txt
478074874.txt
478079492.txt
478080154.txt
478082496.txt
478082624.txt
478083326.txt
478083578.txt
478084156.txt
478085262.txt
478086456.txt
478094168.txt
478096780.txt
478098648.txt
478098770.txt
478100790.txt
478103562.txt
478108324.txt
478110292.txt
478110710.txt
478110986.txt
478488010.txt
478489604.txt
478490330.txt
478490426.txt
478490688.txt
478490806.txt
478490932.txt
478490944.txt
478491182.txt
478493726.txt
478494108.txt
478513056.txt
478514498.txt
478528392.txt
478528404.txt
478535276.txt
478535

486435886.txt
486435922.txt
486436472.txt
486436810.txt
486436982.txt
486438976.txt
486439050.txt
486439058.txt
486439158.txt
486439180.txt
486439654.txt
486440974.txt
486442184.txt
486442406.txt
486442632.txt
486442782.txt
486442914.txt
486442922.txt
486442924.txt
486442926.txt
486443700.txt
486444292.txt
486445084.txt
486446774.txt
486448318.txt
486452780.txt
486452876.txt
486453682.txt
486453684.txt
486454458.txt
486454852.txt
486455780.txt
486455838.txt
486456024.txt
486456046.txt
487986840.txt
487986864.txt
487986890.txt
487988466.txt
487996482.txt
487997078.txt
488048120.txt
488060168.txt
488068864.txt
97924048.txt


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1405: invalid start byte

In [285]:
lin=""
with open("230292662.txt",encoding="utf8") as input_data:
    for line in input_data:
        for line in input_data:
            if any(x in line.lower() for x in ["consolidated statements","consolidated statement","condensed","statement","data"]):
                if any(x in line.lower() for x in ["operations","income","financial","earnings"]):
                    if line.strip()[0].isupper():
                        #if line.strip().istitle():
                        print("upper",line)
                        #if line.strip().istitle():
                        lin+=line
                        break
        for line in input_data:
            if any(x in line.lower() for x in ["months ended","year ended"]):
                if any(x in line.lower() for x in ["january","febraury","march","april","may","june","july","august","september","october","november","december"]):
                    print("lower",line)
                    lin+=line
                    break

upper ITEM 9.01 Financial Statements and Exhibits.

lower today announced operating results for the quarter and year ended December 31, 2017.

upper Supplemental  data  to this  release can  be found  under  the "Financial  Results" navigation  tab on  the "For

lower                                                    Three months ended December 31,        Year ended December 31,

upper CONSOLIDATED STATEMENTS OF OPERATIONS

lower                                                                  Three months ended December 31,        Year ended December 31,

upper  Financial Statements in MAA's Annual Report on Form 10-K for the year ended December 31, 2017, expected to be filed with the SEC on or about February

lower                                      As of December 31, 2017                                 Three Months Ended



In [ ]:
lin=""
with open("230292662.txt",encoding="utf8") as input_data:
    for line in input_data:
        for line in input_data:
            if any(x in line.lower() for x in ["consolidated statements","consolidated statement","condensed","statement","data"]):
                if any(x in line.lower() for x in ["operations","income","financial","earnings"]):
                    if line.strip()[0].isupper():
                        #if line.strip().istitle():
                        print("upper",line)
                        #if line.strip().istitle():
                        lin+=line
                        break
        for line in input_data:
            if any(x in line.lower() for x in ["months ended","year ended"]):
                if any(x in line.lower() for x in ["january","febraury","march","april","may","june","july","august","september","october","november","december"]):
                    print("lower",line)
                    lin+=line
                    break

In [286]:
"<FORM-TYPE>N-14 8C/A".startswith("<FORM-TYPE>")

True

In [268]:
lin

'Included in Other non-operating income (expense) on the Consolidated Statements of Operations\n                                                                                        Months Ended\n'

In [250]:
mv_df_inc.incomeflag.value_counts()

True     1715
False      84
Name: incomeflag, dtype: int64

In [251]:
mv_df_inc["versionid"]=pd.to_numeric(mv_df_inc["versionid"])

In [252]:
mv_main_inc_df=pd.merge(mv_df_inc,sec_metadata_final[["VersionId","Has Income Statement Flag"]],left_on = ['versionid'], right_on = ['VersionId'])

In [220]:
mv_main_inc_df.head(3)

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
0,230288860,"Hillenbrand, Inc.",True,230288860,True
1,230292662,MID AMERICA APARTMENT COMMUNITIES INC,True,230292662,True
2,230292662,MID AMERICA APARTMENT COMMUNITIES INC,True,230292662,True


In [239]:
mv_main_inc_df.shape

(1625, 5)

In [253]:
mv_main_inc_df[mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"]].shape

(84, 5)

In [254]:
mv_main_inc_df[mv_main_inc_df["Has Income Statement Flag"]==mv_main_inc_df["incomeflag"]].shape

(1541, 5)

In [255]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==True)].shape

(73, 5)

In [272]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==False)].shape

(7, 5)

In [257]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==True)]

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
3,230292662,MID AMERICA APARTMENT COMMUNITIES INC,True,230292662,False
9,259118202,Fortive Corp,True,259118202,False
16,273075380,"KBS Strategic Opportunity REIT, Inc.",True,273075380,False
40,342574798,"KBS Strategic Opportunity REIT, Inc.",True,342574798,False
179,463711944,RFPC Holding Corp.,True,463711944,False
192,466354734,VONAGE HOLDINGS CORP,True,466354734,False
197,467467766,"Del Frisco's Restaurant Group, Inc.",True,467467766,False
231,467859448,"General Motors Financial Company, Inc.",True,467859448,False
246,467878752,"Dolphin Entertainment, Inc.",True,467878752,False
260,468459580,HSBC HOLDINGS PLC,True,468459580,False


In [224]:
mv_main_inc_df.incomeflag.value_counts()

True     1428
False     193
Name: incomeflag, dtype: int64

In [225]:
mv_main_inc_df["Has Income Statement Flag"].value_counts()

True     1534
False      91
Name: Has Income Statement Flag, dtype: int64

In [245]:
mv_main_inc_df[mv_main_inc_df.versionid==332386300]

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
27,332386300,LEGACY RESERVES LP,False,332386300,True


In [198]:
sec_metadata_final[sec_metadata_final.VersionId==230292662]

,DocumentId,doc cnt,VersionId,CompanyId,Company Name,documentToObjectRelId,elementGroupId,PeriodOfReport,Language,FilingDate,FilingDateSource,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Is Errata Amendment Flag,Has Notes Flag,Has Audited Financials Flag,Category
647,230292326,3,230292662,332358,"Mid-America Apartment Communities, Inc.",321328575,-2139684062,2017-12-31,English,2018-01-31 16:36:22,Filing date on the website,Annual,True,1.0,0.0,0.0,NaN,NaN,TEST
648,230292326,3,230292662,332358,"Mid-America Apartment Communities, Inc.",321328575,-1415622197,2017-12-31,English,2018-01-31 16:36:22,Filing date on the website,Quarter,True,1.0,0.0,0.0,NaN,NaN,TRAIN
649,230292326,3,230292662,27525007,"Mid-America Apartments, L.P.",321328577,-1415622195,NaT,English,2018-01-31 16:36:22,Filing date on the website,NaN,False,NaN,NaN,NaN,NaN,NaN,VALIDATION


In [226]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==True)]

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
3,230292662,MID AMERICA APARTMENT COMMUNITIES INC,True,230292662,False
9,259118202,Fortive Corp,True,259118202,False
16,273075380,"KBS Strategic Opportunity REIT, Inc.",True,273075380,False
179,463711944,RFPC Holding Corp.,True,463711944,False
192,466354734,VONAGE HOLDINGS CORP,True,466354734,False
195,466988160,ENDEAVOUR SILVER CORP,True,466988160,False
197,467467766,"Del Frisco's Restaurant Group, Inc.",True,467467766,False
231,467859448,"General Motors Financial Company, Inc.",True,467859448,False
259,468457282,WOORI BANK,True,468457282,False
260,468459580,HSBC HOLDINGS PLC,True,468459580,False


In [51]:
sec_metadata[sec_metadata.VersionId==230292662]

,DocumentId,doc cnt,VersionId,CompanyId,Company Name,documentToObjectRelId,elementGroupId,PeriodOfReport,Language,FilingDate,FilingDateSource,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Is Errata Amendment Flag,Has Notes Flag,Has Audited Financials Flag,Category
645,230292326,3,230292662,332358,"Mid-America Apartment Communities, Inc.",321328575,-2139684062,2017-12-31,English,2018-01-31 16:36:22,Filing date on the website,Annual,True,1.0,0.0,0.0,NaN,NaN,TEST
646,230292326,3,230292662,332358,"Mid-America Apartment Communities, Inc.",321328575,-1415622197,2017-12-31,English,2018-01-31 16:36:22,Filing date on the website,Quarter,True,1.0,0.0,0.0,NaN,NaN,TRAIN
647,230292326,3,230292662,27525007,"Mid-America Apartments, L.P.",321328577,-1415622195,NaT,English,2018-01-31 16:36:22,Filing date on the website,NaN,False,NaN,NaN,NaN,NaN,NaN,VALIDATION


In [93]:
sec_metadata_final[sec_metadata_final.VersionId==273075380]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
662,273075380,324286036,KBS SOR (BVI) Holdings Ltd.,2017-12-31,Annual,True,1.0,0.0,NaN
663,273075380,324286036,KBS SOR (BVI) Holdings Ltd.,2018-03-12,Quarter,False,1.0,0.0,NaN


In [105]:
sec_metadata_final[sec_metadata_final.VersionId==273075380]

,VersionId,CompanyId,Company Name,PeriodOfReport,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Has Audited Financials Flag
662,273075380,324286036,KBS SOR (BVI) Holdings Ltd.,2017-12-31,Annual,True,1.0,0.0,NaN


In [94]:
sec_metadata[sec_metadata.VersionId==273075380]

,DocumentId,doc cnt,VersionId,CompanyId,Company Name,documentToObjectRelId,elementGroupId,PeriodOfReport,Language,FilingDate,FilingDateSource,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Is Errata Amendment Flag,Has Notes Flag,Has Audited Financials Flag,Category
661,273074420,3,273075380,52056017,"KBS Strategic Opportunity REIT, Inc.",327368885,-1384960053,NaT,English,2018-03-12 14:07:12,Filing date on the website,NaN,False,NaN,NaN,NaN,NaN,NaN,VALIDATION
662,273074420,3,273075380,324286036,KBS SOR (BVI) Holdings Ltd.,327415815,-1384660709,2017-12-31,English,2018-03-12 14:07:12,Filing date on the website,Annual,True,1.0,0.0,0.0,NaN,NaN,TRAIN
663,273074420,3,273075380,324286036,KBS SOR (BVI) Holdings Ltd.,327415815,-1384659949,2018-03-12,English,2018-03-12 14:07:12,Filing date on the website,Quarter,False,1.0,0.0,NaN,NaN,NaN,TRAIN


In [92]:
mv_main_inc_df[mv_main_inc_df.versionid==273075380]

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
15,273075380,"KBS Strategic Opportunity REIT, Inc.",True,273075380,True
16,273075380,"KBS Strategic Opportunity REIT, Inc.",True,273075380,False


In [ ]:
pd.concat()